## Simulation of the Scenarios

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from scipy.linalg import solve
from scipy.interpolate import BSpline
from scipy.special import erf
import gurobipy as gp
from gurobipy import GRB
from sklearn.model_selection import train_test_split

# Set a random seed for reproducibility
np.random.seed(0)

def run_simulation(g):
    n = 500  # Define the number of samples

    # Simulate data from uniform distributions
    x1 = np.random.uniform(-1, 1, n)
    x2 = np.random.uniform(0, 1, n)
    x3 = np.random.uniform(-1, 1, n)
    x4 = np.random.uniform(0, 1, n)
    x5 = np.random.uniform(-1, 1, n)
    x6 = np.random.uniform(-1, 1, n)
    x_uniform = np.random.uniform(-1, 1, (n, 14))  # Uniform data with 14 features

    # Define several functions f1 to f6
    def f1_def(x):
        return  2*(np.cos( 2* np.pi * np.sqrt((x - 0.5) ** 2))).flatten()

    def f2_def(x: np.ndarray) -> np.ndarray:
        return 9 / (1 + np.exp(-10 * x))  # Logistic function

    def f3_def(x: np.ndarray) -> np.ndarray:
        return 3 * x**3  # Cubic transformation

    def f4_def(x: np.ndarray) -> np.ndarray:
        return x ** 3 + 1.5 * x ** 2  # Cubic plus quadratic

    def f5_def(x: np.ndarray) -> np.ndarray:
        return np.cos(6 * x - 6) - 1.25 * x  # Combination of cosine and linear terms

    def f6_def(x: np.ndarray) -> np.ndarray:
        return 1.6 * x  # Linear transformation

    # Center the functions by subtracting their mean
    f1 = f1_def(x1) - np.mean(f1_def(x1))
    f2 = f2_def(x2) - np.mean(f2_def(x2))
    f3 = f3_def(x3) - np.mean(f3_def(x3))
    f4 = f4_def(x4) - np.mean(f4_def(x4))
    f5 = f5_def(x5) - np.mean(f5_def(x5))
    f6 = f6_def(x6) - np.mean(f6_def(x6))

    # Construct the additive model
    f_sum = f1 + f2 + f3 + f4 + f5 + f6   
    variance = g * np.var(f_sum)  
    y = f_sum + np.random.normal(0, np.sqrt(variance), n)  # Add noise to the signal
    
    # Split the data into training and testing sets
    train_size = 400
    test_size = n - train_size

    indices = np.arange(n)
    train_indices, test_indices = train_test_split(indices, train_size=train_size, shuffle=False, random_state=0)

    x1_train, x1_test = x1[train_indices], x1[test_indices]
    x2_train, x2_test = x2[train_indices], x2[test_indices]
    x3_train, x3_test = x3[train_indices], x3[test_indices]
    x4_train, x4_test = x4[train_indices], x4[test_indices]
    x5_train, x5_test = x5[train_indices], x5[test_indices]
    x6_train, x6_test = x6[train_indices], x6[test_indices]
    x_uniform_train, x_uniform_test = x_uniform[train_indices], x_uniform[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]

    # Generate extended range for theorical curve looks smoother 
    x1_train_ext = np.linspace(np.min(x1_train), np.max(x1_train), train_size)
    x2_train_ext = np.linspace(np.min(x2_train), np.max(x2_train), train_size)
    x3_train_ext = np.linspace(np.min(x3_train), np.max(x3_train), train_size)
    x4_train_ext = np.linspace(np.min(x4_train), np.max(x4_train), train_size)
    x5_train_ext = np.linspace(np.min(x5_train), np.max(x5_train), train_size)
    x6_train_ext = np.linspace(np.min(x6_train), np.max(x6_train), train_size)

    # Compute training functions based on the training data
    f1_train = f1_def(x1_train) - np.mean(f1_def(x1_train))
    f2_train = f2_def(x2_train) - np.mean(f2_def(x2_train))
    f3_train = f3_def(x3_train) - np.mean(f3_def(x3_train))
    f4_train = f4_def(x4_train) - np.mean(f4_def(x4_train))
    f5_train = f5_def(x5_train) - np.mean(f5_def(x5_train))
    f6_train = f6_def(x6_train) - np.mean(f6_def(x6_train))

    # Compute training extended functions for extended ranges
    f1_train_ext = f1_def(x1_train_ext) - np.mean(f1_def(x1_train_ext))
    f2_train_ext = f2_def(x2_train_ext) - np.mean(f2_def(x2_train_ext))
    f3_train_ext = f3_def(x3_train_ext) - np.mean(f3_def(x3_train_ext))
    f4_train_ext = f4_def(x4_train_ext) - np.mean(f4_def(x4_train_ext))
    f5_train_ext = f5_def(x5_train_ext) - np.mean(f5_def(x5_train_ext))
    f6_train_ext = f6_def(x6_train_ext) - np.mean(f6_def(x6_train_ext))

    K = 6  # Number of selected covariates
    num_nodes = 20  # Number of nodes for B-splines
    lambdas = np.logspace(-4, 4, 100)  # Lambda values for regularization

    # Define constraints for optimization depending on the Scenario
    constraints = []    # Scenario 1: No constraints
    #constraints = ['f2_concavity', 'f3_monotonicity', 'f4_convexity']    # Scenario 2
    #constraints = ['f2_concavity', 'f3_monotonicity', 'f4_convexity', 'x8_monotonicity_decrec', 'x15_concavity','x19_convexity']  # Scenario 3
    #constraints = ['x8_monotonicity_decrec', 'x15_concavity','x19_convexity']   # Scenario 4

    # Function to find the optimal lambda using generalized cross-validation (GCV)
    def optimal_lambda_gcv(B, y, lambdas):
        D = np.diff(np.diag(np.ones(B.shape[1])), n=2).T  
        n, m = B.shape
        gcv_values = []
        for lambd in lambdas:
            P = lambd * np.dot(np.transpose(D), D)  
            H = B @ solve(B.T @ B + P, B.T) 
            y_hat = H @ y  # Predicted values
            residuals = y - y_hat  # Residuals
            h = np.diag(H)  # Diagonal elements of H
            gcv = np.sum((residuals ** 2) / ((1 - h) ** 2)) / n  # GCV criterion
            gcv_values.append(gcv)

        gcv_values = np.array(gcv_values)
        optimal_lambda = lambdas[np.argmin(gcv_values)]  # Select lambda that minimizes GCV

        return optimal_lambda, gcv_values

    # Optimization function 
    def optimization_function(x1_train, x2_train, x3_train, x4_train, x5_train, x6_train, x_uniform_train,
                              x1_train_ext, x2_train_ext, x3_train_ext, x4_train_ext, x5_train_ext, x6_train_ext,
                              x1_test, x2_test, x3_test, x4_test, x5_test, x6_test, x_uniform_test, 
                              y_train, y_test, K, num_nodes, lambdas, constraints):

        # B-spline basis function generator
        def my_bbase(x, xl, xr, ndx=10, bdeg=3, eps=1e-05):
            dx = (xr - xl) / ndx
            knots = np.linspace(xl - bdeg * dx, xr + bdeg * dx, ndx + 2 * bdeg)  # Knot sequence
            c = np.eye(len(knots) - bdeg) 
            B = BSpline(knots, c, bdeg)(x)  # Evaluate the spline basis at points x
            return B

        # Create B-spline basis for each covariable
        B1 = my_bbase(x1_train, np.min(x1_train), np.max(x1_train), num_nodes, 3).squeeze()
        B2 = my_bbase(x2_train, np.min(x2_train), np.max(x2_train), num_nodes, 3).squeeze()
        B3 = my_bbase(x3_train, np.min(x3_train), np.max(x3_train), num_nodes, 3).squeeze()
        B4 = my_bbase(x4_train, np.min(x4_train), np.max(x4_train), num_nodes, 3).squeeze()
        B5 = my_bbase(x5_train, np.min(x5_train), np.max(x5_train), num_nodes, 3).squeeze()
        B6 = my_bbase(x6_train, np.min(x6_train), np.max(x6_train), num_nodes, 3).squeeze()
        B_uniform = np.hstack([my_bbase(x_uniform_train[:, i], -1, 1 , num_nodes, 3).squeeze() for i in range(14)])
       
        B1_ext = my_bbase(x1_train_ext, np.min(x1_train_ext), np.max(x1_train_ext), num_nodes, 3).squeeze()
        B2_ext = my_bbase(x2_train_ext, np.min(x2_train_ext), np.max(x2_train_ext), num_nodes, 3).squeeze()
        B3_ext = my_bbase(x3_train_ext, np.min(x3_train_ext), np.max(x3_train_ext), num_nodes, 3).squeeze()
        B4_ext = my_bbase(x4_train_ext, np.min(x4_train_ext), np.max(x4_train_ext), num_nodes, 3).squeeze()
        B5_ext = my_bbase(x5_train_ext, np.min(x5_train_ext), np.max(x5_train_ext), num_nodes, 3).squeeze()
        B6_ext = my_bbase(x6_train_ext, np.min(x6_train_ext), np.max(x6_train_ext), num_nodes, 3).squeeze()

        B1_test = my_bbase(x1_test, np.min(x1_train), np.max(x1_train), num_nodes, 3).squeeze()
        B2_test = my_bbase(x2_test, np.min(x2_train), np.max(x2_train), num_nodes, 3).squeeze()
        B3_test = my_bbase(x3_test, np.min(x3_train), np.max(x3_train), num_nodes, 3).squeeze()
        B4_test = my_bbase(x4_test, np.min(x4_train), np.max(x4_train), num_nodes, 3).squeeze()
        B5_test = my_bbase(x5_test, np.min(x5_train), np.max(x5_train), num_nodes, 3).squeeze()
        B6_test = my_bbase(x6_test, np.min(x6_train), np.max(x6_train), num_nodes, 3).squeeze()
        B_uniform_test = np.hstack([my_bbase(x_uniform_test[:, i], -1, 1, num_nodes, 3).squeeze() for i in range(14)])

        # Select the optimal lambda for each of the B matrices using GCV
        lambda1, _ = optimal_lambda_gcv(B1, y_train, lambdas)
        lambda2, _ = optimal_lambda_gcv(B2, y_train, lambdas)
        lambda3, _ = optimal_lambda_gcv(B3, y_train, lambdas)
        lambda4, _ = optimal_lambda_gcv(B4, y_train, lambdas)
        lambda5, _ = optimal_lambda_gcv(B5, y_train, lambdas)
        lambda6, _ = optimal_lambda_gcv(B6, y_train, lambdas)
        lambda_uniform_list = [optimal_lambda_gcv(B_uniform[:, i * 23:(i + 1) * 23], y_train, lambdas)[0] for i in range(14)]

        # Calculate penalty matrices for each basis
        D1 = np.diff(np.diag(np.ones(B1.shape[1])), n=2).T
        P1 = lambda1 * np.dot(np.transpose(D1), D1)
        D2 = np.diff(np.diag(np.ones(B2.shape[1])), n=2).T
        P2 = lambda2 * np.dot(np.transpose(D2), D2)
        D3 = np.diff(np.diag(np.ones(B3.shape[1])), n=2).T
        P3 = lambda3 * np.dot(np.transpose(D3), D3)
        D4 = np.diff(np.diag(np.ones(B4.shape[1])), n=2).T
        P4 = lambda4 * np.dot(np.transpose(D4), D4)
        D5 = np.diff(np.diag(np.ones(B5.shape[1])), n=2).T
        P5 = lambda5 * np.dot(np.transpose(D5), D5)
        D6 = np.diff(np.diag(np.ones(B6.shape[1])), n=2).T
        P6 = lambda6 * np.dot(np.transpose(D6), D6)

        # Create a column vector of ones
        X = np.ones((len(x1_train), 1))

        # For identifiability issue
        PP1 = P1 + np.dot(np.dot(np.dot(B1.T, X), X.T), B1)
        PP2 = P2 + np.dot(np.dot(np.dot(B2.T, X), X.T), B2)
        PP3 = P3 + np.dot(np.dot(np.dot(B3.T, X), X.T), B3)
        PP4 = P4 + np.dot(np.dot(np.dot(B4.T, X), X.T), B4)
        PP5 = P5 + np.dot(np.dot(np.dot(B5.T, X), X.T), B5)
        PP6 = P6 + np.dot(np.dot(np.dot(B6.T, X), X.T), B6)

        PP_uniform_list = []
        for i in range(14):
            start_col = i * 23
            end_col = start_col + 23
            B_uni = B_uniform[:, start_col:end_col]
            D_uniform = np.diff(np.diag(np.ones(B_uni.shape[1])), n=2).T
            P_uniform = lambda_uniform_list[i] * np.dot(np.transpose(D_uniform), D_uniform)
            PP_uni = P_uniform + np.dot(np.dot(np.dot(B_uni.T, X), X.T), B_uni)
            PP_uniform_list.append(PP_uni)

        # Dimensions of penalty matrices
        n1, m1 = PP1.shape
        n2, m2 = PP2.shape
        n3, m3 = PP3.shape
        n4, m4 = PP4.shape
        n5, m5 = PP5.shape
        n6, m6 = PP6.shape
        n_uniform = [P.shape[0] for P in PP_uniform_list]
        m_uniform = [P.shape[1] for P in PP_uniform_list]
        
        total_n = 1 + n1 + n2 + n3 + n4 + n5 + n6 + sum(n_uniform)
        total_m = 1 + m1 + m2 + m3 + m4 + m5 + m6 + sum(m_uniform)

        # Create the complete penalty matrix PP
        PP = np.zeros((total_n, total_m))

        PP[1:1 + n1, 1:1 + m1] = PP1
        PP[1 + n1:1 + n1 + n2, 1 + m1:1 + m1 + m2] = PP2
        PP[1 + n1 + n2:1 + n1 + n2 + n3, 1 + m1 + m2:1 + m1 + m2 + m3] = PP3
        PP[1 + n1 + n2 + n3:1 + n1 + n2 + n3 + n4, 1 + m1 + m2 + m3 + m4:1 + m1 + m2 + m3 + m4 + m5] = PP4
        PP[1 + n1 + n2 + n3 + n4:1 + n1 + n2 + n3 + n4 + n5, 1 + m1 + m2 + m3 + m4 + m5:1 + m1 + m2 + m3 + m4 + m5 + m6] = PP5
        PP[1 + n1 + n2 + n3 + n4 + n5:1 + n1 + n2 + n3 + n4 + n5 + n6, 1 + m1 + m2 + m3 + m4 + m5 + m6:1 + m1 + m2 + m3 + m4 + m5 + m6 + m6] = PP6

        start_idx_n = 1 + n1 + n2 + n3 + n4 + n5 + n6
        start_idx_m = 1 + m1 + m2 + m3 + m4 + m5 + m6
        for i, PP_uniform in enumerate(PP_uniform_list):
            n, m = PP_uniform.shape
            PP[start_idx_n:start_idx_n + n, start_idx_m:start_idx_m + m] = PP_uniform
            start_idx_n += n
            start_idx_m += m

        # Create the complete B matrices with all predictor variables
        k = B1.shape[1] + B2.shape[1] + B3.shape[1] + B4.shape[1] + B5.shape[1] + B6.shape[1] + B_uniform.shape[1] - 3
        B = np.hstack((X, B1, B2, B3, B4, B5, B6, B_uniform))
        B_test = np.hstack((B1_test, B2_test, B3_test, B4_test, B5_test, B6_test, B_uniform_test))
        B_ext = np.hstack((B1_ext, B2_ext, B3_ext, B4_ext, B5_ext, B6_ext, B_uniform))
        
        # Create the model
        model = gp.Model("ConstrainedOptimization")

        # Create decision variables
        theta = {j: model.addVar(lb=-GRB.INFINITY, name=f"theta_{j}") for j in range(k + 4)}
        s = {l: model.addVar(vtype=GRB.BINARY, name=f"s_{l}") for l in range(20)}

        # Add monotonicity and concavity constraints based on the list of constraints
        if 'f2_concavity' in constraints:
            for i in range(B2.shape[1] - 2):
                model.addConstr(theta[1 + B1.shape[1] + i + 2] - 2 * theta[1 + B1.shape[1] + i + 1] + theta[1 + B1.shape[1] + i] <= 0, name=f"f2_concavity_{i}")
        if 'f3_monotonicity' in constraints:
            for i in range(B3.shape[1] - 1):
                model.addConstr(theta[1 + B1.shape[1]+ B2.shape[1] + i + 1] - theta[1 + B1.shape[1] + B2.shape[1] + i] >= 0, name=f"f3_monotonicity_{i}")
        if 'f4_convexity' in constraints:
            for i in range(B4.shape[1] - 2):
                model.addConstr(theta[1 + B1.shape[1] + B2.shape[1] + B3.shape[1] + i + 2] - 2 * theta[1 + B1.shape[1] + B2.shape[1] + B3.shape[1] + i + 1] + theta[1 + B1.shape[1] + B2.shape[1] + B3.shape[1] + i] >= 0, name=f"f4_convexity_{i}")
        if 'x8_monotonicity_decrec' in constraints: 
            for i in range(B1.shape[1] - 1):
                model.addConstr(theta[1 + 7 * B1.shape[1] +i + 1] - theta[1 + 7 * B1.shape[1] + i] <= 0, name=f"x8_monotonicity_decrec_{i}")
        if 'x15_concavity' in constraints:
            for i in range(B1.shape[1] - 2):
                model.addConstr(theta[1 + 14 * B1.shape[1] + i + 2] - 2 * theta[1 + 14 * B1.shape[1] + i + 1] + theta[1 + 14 * B1.shape[1] + i] <= 0, name=f"x15_concavity_{i}")     
        if 'x19_convexity' in constraints:
            for i in range(B1.shape[1] - 2):
                model.addConstr(theta[1 + 18 * B1.shape[1] + i + 2] - 2 * theta[1 + 18 * B1.shape[1] + i + 1] + theta[1 + 18 * B1.shape[1] + i] >= 0, name=f"x19_convexity_{i}")      

        # Objective function
        obj = gp.quicksum((y_train[i] - gp.quicksum(B[i][j] * theta[j] for j in range(k + 4))) ** 2 for i in range(len(y_train)))
        penalization = gp.quicksum(theta[i] * PP[i, j] * theta[j] for j in range(PP.shape[1]) for i in range(PP.shape[0]))
        obj += penalization
        model.setObjective(obj, GRB.MINIMIZE)

        # Variable selection constraint
        model.addConstr(gp.quicksum(s[l] for l in range(20)) <= K)

        M = 1000

        for l in range(20):
            for m_l in range(B1.shape[1]):
                model.addConstr(-M * s[l] <= theta[1 + B1.shape[1] * l + m_l], name=f"constraint1_{l}_{m_l}")
                model.addConstr(theta[1 + B1.shape[1] * l + m_l] <= M * s[l], name=f"constraint2_{l}_{m_l}")

        # Optimize the model
        model.optimize()
        
        if model.status == GRB.OPTIMAL:
            opt_theta = [theta[j].X for j in range(k + 4)]
            
            # Calculate the predictions 
            y_pred = np.dot(B_test, opt_theta[1:])
            
            # Compute the MAE and MSE
            mae = np.mean(np.abs(y_pred - y_test))
            mse = np.mean((y_pred - y_test) ** 2)
            
            # Calculate f(x) estimates for each data set
            f1_est = np.dot(B1_ext, opt_theta[1:1 + B1.shape[1]])
            f2_est = np.dot(B2_ext, opt_theta[1 + B1.shape[1]:1 + B1.shape[1] + B2.shape[1]])
            f3_est = np.dot(B3_ext, opt_theta[1 + B1.shape[1] + B2.shape[1]:1 + B1.shape[1] + B2.shape[1] + B3.shape[1]])
            f4_est = np.dot(B4_ext, opt_theta[1 + B1.shape[1] + B2.shape[1] + B3.shape[1]:1 + B1.shape[1] + B2.shape[1] + B3.shape[1] + B4.shape[1]])
            f5_est = np.dot(B5_ext, opt_theta[1 + B1.shape[1] + B2.shape[1] + B3.shape[1] + B4.shape[1]:1 + B1.shape[1] + B2.shape[1] + B3.shape[1] + B4.shape[1] + B5.shape[1]])
            f6_est = np.dot(B6_ext, opt_theta[1 + B1.shape[1] + B2.shape[1] + B3.shape[1] + B4.shape[1] + B5.shape[1]:1 + B1.shape[1] + B2.shape[1] + B3.shape[1] + B4.shape[1] + B5.shape[1] + B6.shape[1]])
        
            solution = {
                "status": "Optimal solution found", 
                "theta": {j: theta[j].X for j in range(k + 4)}, 
                "s": {l: s[l].X for l in range(20)}, 
                "mse": mse ,
                "mae": mae, 
                "x_train": [x1_train, x2_train, x3_train, x4_train, x5_train, x6_train],
                "x_train_ext": [x1_train_ext, x2_train_ext, x3_train_ext, x4_train_ext, x5_train_ext, x6_train_ext],
                "f_train": [f1_train, f2_train, f3_train, f4_train, f5_train, f6_train],
                "f_train_ext": [f1_train_ext, f2_train_ext, f3_train_ext, f4_train_ext, f5_train_ext, f6_train_ext],
                "f_est": [f1_est, f2_est, f3_est, f4_est, f5_est, f6_est]
             }

        else:
            solution = {"status": "No optimal solution found"}

        return solution

    solution = optimization_function(x1_train, x2_train, x3_train, x4_train, x5_train, x6_train, x_uniform_train, x1_train_ext, x2_train_ext, x3_train_ext, x4_train_ext, x5_train_ext, x6_train_ext, x1_test, x2_test, x3_test, x4_test, x5_test, x6_test, x_uniform_test, y_train, y_test, K, num_nodes, lambdas, constraints)
    return solution

In [ ]:
# The value of the SNR depends on the value of g, for each Scenario
g = 1         #SNR=1
#g = 0.5      #SNR=2
#g = 0.25     #SNR=4

results_1_1 = [] # Create an empty list to store the results
# Run a loop for 50 iterations
for i in range(50):
    print('Simulation:',i+1)
    results_1_1.append(run_simulation(g))

In [ ]:
# Metrics of the performance of the model:
# Initialize empty lists to store metrics for each simulation
tpr_list = []  
tnr_list = []  
acc_list = [] 
mae_list = []  
mse_list = []  


for i, result in enumerate(results_1_1):
    if result["status"] == "Optimal solution found":
        # Get the variables selected by the model and error metrics
        s_selected = result["s"]  # Selected variables by the model
        mae = result["mae"]  # Mean Absolute Error
        mse = result["mse"]  # Mean Squared Error
        
        # Initialize counters for TPR, TNR, and Accuracy
        true_positives = 0  
        false_negatives = 0  
        true_negatives = 0  
        false_positives = 0  
        
        for j in range(6):
            if s_selected[j] == 1:
                true_positives += 1
            else:
                false_negatives += 1
        
        for j in range(6, 20):
            if s_selected[j] == 0:
                true_negatives += 1
            else:
                false_positives += 1
                
        total_positives = true_positives + false_negatives
        total_negatives = true_negatives + false_positives
        
        # Calculate True Positive Rate (TPR) 
        if total_positives > 0:
            tpr = true_positives / 6  # Divide by 6, the total number of relevant variables
        else:
            tpr = 0.0 
        
        # Calculate True Negative Rate (TNR) 
        if total_negatives > 0:
            tnr = true_negatives / 14  # Divide by 14, the total number of irrelevant variables
        else:
            tnr = 0.0 
        
        # Calculate accuracy as the proportion of correct classifications
        accuracy = (true_positives + true_negatives) / 20.0  # 20 variables in total (6 relevant + 14 irrelevant)
        
        # Append calculated metrics to their respective lists
        tpr_list.append(tpr)
        tnr_list.append(tnr)
        acc_list.append(accuracy)
        mae_list.append(mae)
        mse_list.append(mse)
        
    else:
        print(f"No optimal solution found for Simulation {i+1}")

# Calculate the mean and standard deviation of the metrics
tpr_mean = np.mean(tpr_list)  
tpr_std = np.std(tpr_list)  

tnr_mean = np.mean(tnr_list)  
tnr_std = np.std(tnr_list)  

acc_mean = np.mean(acc_list)  
acc_std = np.std(acc_list)  

mae_mean = np.mean(mae_list)  
mae_std = np.std(mae_list) 

mse_mean = np.mean(mse_list)  
mse_std = np.std(mse_list) 

# Create a DataFrame to summarize the mean and standard deviation of the metrics
summary_df_1_1 = pd.DataFrame({
    "Metric": ["TPR", "TNR", "ACC", "MAE", "MSE"],
    "Mean": [tpr_mean, tnr_mean, acc_mean, mae_mean, mse_mean],
    "Standard Deviation": [tpr_std, tnr_std, acc_std, mae_std, mse_std]
})

print("Summary of Results:")
print(summary_df_1_1)

In [ ]:
# # Plot the result of the simulation with the lowest MSE
min_mse = float('inf')  
best_result_1_1 = None  


for result in results_1_1:
    # Check if the simulation found an optimal solution and has a lower MSE than the current minimum
    if result["status"] == "Optimal solution found" and result["mse"] < min_mse:
        min_mse = result["mse"]  
        best_result_1_1 = result  # Store the result with the lowest MSE


if best_result_1_1 is not None:
    # Extract variables from the best simulation result
    opt_theta = best_result_1_1["theta"]  
    x_train_list = best_result_1_1["x_train"]  
    x_train_ext_list = best_result_1_1["x_train_ext"]  
    f_train_list = best_result_1_1["f_train"]  
    f_train_ext_list = best_result_1_1["f_train_ext"] 
    f_est_list = best_result_1_1["f_est"]  

    # Print the MSE of the best simulation
    print(best_result_1_1["mse"])

    plt.figure(figsize=(15, 10))
    
    for i, (x_train, x_train_ext, f_train, f_train_ext, f_est, title) in enumerate(zip(
            x_train_list, x_train_ext_list, f_train_list, f_train_ext_list, f_est_list,
            ['Predicted Curve 1 vs. Simulated Data', 'Predicted Curve 2 vs. Simulated Data',
             'Predicted Curve 3 vs. Simulated Data', 'Predicted Curve 4 vs. Simulated Data',
             'Predicted Curve 5 vs. Simulated Data', 'Predicted Curve 6 vs. Simulated Data']
        )):
        plt.subplot(2, 3, i + 1)
        
        # Scatter plot of the training data (blue dots)
        plt.scatter(x_train, f_train, color='blue', label=f'Data Set {i + 1}')
        
        # Plot of the theoretical curve (black line)
        plt.plot(x_train_ext, f_train_ext, color='black', label=f'Theoretical Curve {i + 1}')
        
        # Plot of the model's predicted curve (red line)
        plt.plot(x_train_ext, f_est, color='red', linestyle='-', label=f'Predictions {i + 1}')
        
        plt.xlabel('x')
        plt.ylabel('y')
        plt.title(title)
        plt.legend()
        plt.grid(True)

    plt.tight_layout()
    plt.show()

else:
    print("No optimal solution found in the 50 simulations.")